In [1]:
# GENERAL GATES
measure_gate = 'measure'
reset_gate = 'reset'

# 1. IONQ -> Only valid gates are:
#   - GPI: Puerta IonQ GPi
#   - GPI2: Puerta IonQ GPi2.
#   - MS: Puerta IonQ de Mølmer-Sørenson.

#GPI = Rz(2ϕ)X
#GPI2 = Rz(ϕ)Rx(π/2)Rz(−ϕ)
ionq_predecomposition_gates = ["id", "rx", "ry", "rz", "cx"]
ionq_valid_1qgates = ['gpi', 'gpi2']
ionq_valid_2qgates = ['ms']
ionq_valid_gates = ionq_valid_1qgates + ionq_valid_2qgates

# 2. IBMQ Kyiv -> ECR, ID, RZ, SX, X
ibmq_valid_1qgates = ["id", "rz", "sx", "x"]
ibmq_valid_2qgates = ["ecr"]
ibmq_valid_gates = ibmq_valid_1qgates + ibmq_valid_2qgates

# CIRCUIT TO SIMULATE

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister

def initialize_circuit(qregs = None, n_qubits = None):
    if qregs is None:
        return QuantumCircuit(n_qubits)
    
    qregs_list = qregs.values()
    return QuantumCircuit(*qregs_list)


def get_circuit_max120qubits(qregs = None, n_qubits = None):
    q_circuit = initialize_circuit(qregs, n_qubits)

    q_circuit.h(0)
    for i in range(119):
        q_circuit.cx(i, i+1)
    
    q_circuit.measure_all()
    return q_circuit

In [13]:
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp

n_qubits = 120
 
service = QiskitRuntimeService(channel="ibm_quantum", token="83c50a9eda4c0a420b822eb58610cbbbc9e39e1232e773d5c1cf9de2c06722776875de7e178c01d7d74563775fe117706ffd958f12e84f78a22b8470908e17b3")
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=n_qubits)
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

estimator = Estimator(backend, options={"default_shots": 2000})
isa_circuit = pm.run(get_circuit_max120qubits(n_qubits=120))
job = estimator.run([isa_circuit])

ValueError: An invalid Estimator pub-like was given (<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>). If you want to run a single pub, you need to wrap it with `[]` like `estimator.run([(circuit, observables, param_values)])` instead of `estimator.run((circuit, observables, param_values))`.

In [ ]:
job_result = job.result()

for idx, pub_result in enumerate(job_result):
    print(f"Expectation values for pub {idx}: {pub_result.data.meas.get_counts()}")

AttributeError: 'DataBin' object has no attribute 'meas'